In [147]:
import cv2
import numpy as np
import pandas as pd
import os
import glob
import re
import pickle
from matplotlib import pyplot as plt

## Feature Extraction

#### Baseline

This baseline builds a feature set by pulling pairs of images and creating a difference of the image histograms they produce. The assumption is that files are contained in some input directory in the form patientX-1.jpg, patientX-2.jpg

In [138]:
def hist_features(img_path):
    img = cv2.imread(img_path, 0)
    # Based on my research 256 is the value to use for full range
    hist = cv2.calcHist([img],[0],None,[256],[0,256])
    return hist.flatten()

def baseline_hist_diff(img_before, img_after):
    return hist_features(img_before) - hist_features(img_after)

In [159]:
def extract_features(input_dir):
    img_hist = []
    patient_ids = []
    file_set = glob.glob(os.path.join(input_dir,'patient*-1.jpg'))
    for f in file_set:
        diff = baseline_hist_diff(f, f.replace("-1.jpg", "-2.jpg"))
        img_hist.append(diff)
        
        # Add patient id
        patient_ids.append(re.search('patient(.*)-1.jpg', f).group(1))
    column_names = ["hist" + str(i) for i in range(256)]
    # Construct a pandas dataframe
    return pd.DataFrame(img_hist, columns=column_names, index=patient_ids)

r = extract_features('test_images')

In [160]:
r

,hist0,hist1,hist2,hist3,hist4,hist5,hist6,hist7,hist8,hist9,...,hist246,hist247,hist248,hist249,hist250,hist251,hist252,hist253,hist254,hist255
A,-6310.0,-35807.0,-4816.0,-2605.0,-1620.0,-1083.0,-761.0,-568.0,-467.0,-373.0,...,-200.0,-228.0,-348.0,-489.0,-589.0,-797.0,-1028.0,-1151.0,-1675.0,-4493.0
